<a href="https://colab.research.google.com/github/nguyenanhtienabcd/AIO2024_EXERCISE/blob/feature%2FMODULE7-WEEK2/m07w02_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import các thư viện cần thiết

In [ ]:
pip install kagglehub

In [ ]:
import kagglehub
data_dir = kagglehub.dataset_download("andrewmvd/dog-and-cat-detection")
print('Path to dataset files:', data_dir)

100%|██████████| 1.03G/1.03G [00:21<00:00, 51.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/andrewmvd/dog-and-cat-detection/versions/1


In [ ]:
!kaggle datasets download -d andrewmvd/dog-and-cat-detection -p ./dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/dog-and-cat-detection
License(s): CC-BY-SA-4.0
 99% 1.02G/1.03G [00:09<00:00, 120MB/s]
100% 1.03G/1.03G [00:10<00:00, 110MB/s]


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import seaborn as sns
import torch.optim as optim
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from torchvision.models.resnet import ResNet18_Weights

*   Khởi tạo
*   Lọc dữ liệu
*   Đếm object
*   lấy mẫu
*   phân tích annotations

In [ ]:
from genericpath import isfile
# định nghĩa một lớp class image mới
class ImageDataset(Dataset):

  #------ khởi tạo dữ liệu---------
  def __init__(self, annotations_dir, image_dir, transform = None):
    self.annotations_dir = annotations_dir
    self.image_dir = image_dir
    self.transform = transform
    self.image_files = self.filter_image_with_multiple_objects()

  #------ lọc dữ liệu---------
  def filter_image_with_multiple_objects(self):
    # tạo một list chứa các file thỏa mãn
    valid_image_files = []
    for f in os.listdir(self.image_dir):
      if os.path.isfile(os.path.join(self.image_dir, f)):
        img_name = f
        # mục đích của đoạn code này là chia file thành 2 phần
        # [file name, extention] => [0] là lấy file name
        annotation_name = img_name.split('.')[0] + '.xml'
        annotation_path = os.path.join(self.annotations_dir, annotation_name)

        # tạo một hàm dữ lại các ảnh có object đơn
        if self.count_objects_in_annotation(annotation_path) <=1:
          valid_image_files.append(img_name)
        else:
          print(f'Image {img_name} has multiple objects and will be excluded the dataset')
    return valid_image_files

   #------ đếm đối tượng---------
  def count_objects_in_annotation(self, annotation_path):
    try:
      tree = ET.parse(annotation_path) # đọc file xml từ path
      root = tree.getroot() # lấy phần gốc của file xml
      count = 0
      for obj in root.findall('object'):
        count += 1
      return count
    except Exception as e:
      print(f'Error reading annotation file {annotation_path}: {e}')
      return 0

  def __len__(self):
    return len(self.image_files)

  #------ lấy mẫu---------
  def __getitem__(self, idx):
    # đường dẫn ảnh
    img_name = self.image_files[idx]
    img_path = os.path.join(self.image_dir, img_name)

    # load ảnh lên
    image = Image.open(img_path).convert('RGB')

    # annotation path
    annotation_name = img_name.split('.')[0] + '.xml'
    annotation_path = os.path.join(self.annotations_dir, annotation_name)

    # Parse annotation
    label = self.parse_annotation(annotation_path)
    if self.transform:
      image = self.transform(image)
    return image, label

    #------ phân tích annotations---------
  def parse_annotation(self, annotation_path): # un-indent this method
      tree = ET.parse(annotation_path)
      root = tree.getroot()

      label = None
      for obj in root.findall('object'):
        name = obj.find('name').text
        if (label is None):
          # lấy label đầu tiên
          label = name

        label_num = 0 if label == 'cat' else 1 if label == 'dog' else -1
        return label_num # This return statement may need to be outside the loop.

In [ ]:
# data directory
annotations_dir = os.path.join(data_dir, 'annotations')
image_dir = os.path.join(data_dir, 'images')

# lấy  dánh sách các files và tạo một dạng dữ liệu để chia dữ liệu
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
df = pd.DataFrame({'image_name': image_files})


In [ ]:
# chi tập dữ liệu
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
# transform và chuyển dữ liệu sang tensor
from torchvision import transforms
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset
#train_dataset, val_dataset, test_dataset có cùng một tệp dữ liệu
train_dataset = ImageDataset(annotations_dir, image_dir, transform)
val_dataset = ImageDataset(annotations_dir, image_dir, transform)
test_dataset = ImageDataset(annotations_dir, image_dir, transform)

# lọc dataset
# lọc tất cả các file nào nằm trong dataframe train, val, test
train_dataset.image_files =  [f for f in train_dataset.image_files if f in train_df['image_name'].values]
val_dataset.image_files =  [f for f in val_dataset.image_files if f in val_df['image_name'].values]
test_dataset.image_files =  [f for f in test_dataset.image_files if f in test_df['image_name'].values]




Image Cats_Test736.png has multiple objects and will be excluded the dataset
Image Cats_Test736.png has multiple objects and will be excluded the dataset
Image Cats_Test736.png has multiple objects and will be excluded the dataset


In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

xây dựng model

In [ ]:
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# show model sumary
print(model)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 121MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

training loop

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
  model.train()
  train_loss = 0.0
  for batch_idx, (data, targets) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, targets)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  # validation
  model.eval()
  val_loss = 0.0
  with torch.no_grad():
    for data, targets in val_loader:
      output = model(data)
      loss = criterion(output, targets)
      val_loss += loss.item()

      _,predictions = output.max(1)
      correct = (predictions == targets).sum().item()
      total = targets.size(0)
      accuracy = correct / total
  print(f'Epoch [{epoch+1}/{num_epochs}],Validation Accuracy: {accuracy:.4f}')

Epoch [1/10],Validation Accuracy: 1.0000
Epoch [2/10],Validation Accuracy: 1.0000
Epoch [3/10],Validation Accuracy: 0.5000
Epoch [4/10],Validation Accuracy: 1.0000
Epoch [5/10],Validation Accuracy: 1.0000
Epoch [6/10],Validation Accuracy: 1.0000
Epoch [7/10],Validation Accuracy: 0.5000
Epoch [8/10],Validation Accuracy: 1.0000
